Aim: Filter out the low-quality data

In [7]:
# from ultralytics import YOLO
import pandas as pd
import torch as th
import src.data_loader as dl
# get yolo to detect mosquito
from ultralytics import YOLO
from openmax.clf_utils import calculate_iou
from PIL import Image
from torchvision import transforms
import os

IMG_SIZE = (224, 224)
USE_CHANNEL_LAST = False
DATASET = "laion"
DEVICE = "cuda:0"
PRESERVE_ASPECT_RATIO = False
SHIFT = 0

In [8]:
yolo_path = "/home/pc2/Downloads/AI-Crowd/Mosquito-Classifiction/experiments/yolo/runs/detect/train_6class/weights/best.pt"
CLASS_DICT = {
    "albopictus": th.tensor(0, dtype=th.float),
    "culex": th.tensor(1, dtype=th.float),
    "japonicus/koreicus": th.tensor(2, dtype=th.float),
    #"culiseta": th.tensor([0, 0, 0, 1,], dtype=th.float),
    "aegypti": th.tensor(3, dtype=th.float),
    #"anopheles": th.tensor(4, dtype=th.float),
    # "mosquito": th.tensor(4, dtype=th.float)
}

class_dict = {
"albopictus": th.tensor(0, dtype=th.float),
"culex": th.tensor(1, dtype=th.float),
"japonicus/koreicus": th.tensor(2, dtype=th.float),
"aegypti": th.tensor(3, dtype=th.float),
"mosquito": th.tensor(4, dtype=th.float)
} 
img_dir = "" #"../data_round_2/final/"
test_annotations_csv = "../data_round_2/mosAlert_new_annotation_1/test_annotation_1.csv"
test_df = pd.read_csv(test_annotations_csv)
test_df = test_df.sample(frac=1).reset_index(drop=True) # need shuffle test set


# create test set from dl
test_set = dl.TestYOLOCLIPDataset(annotations_df=test_df, 
                                  class_dict=class_dict,
                                  img_dir=img_dir,
                                #   transform=dl.pre_process(dataset),
                                #   data_augment=dl.aug("resize", img_size),
                                  )

In [9]:
yolo_model = YOLO(
    yolo_path,
    task="detect",
)


In [10]:
def get_bbox(img, yolo_model):
    # detect mosquito
    results = yolo_model(img, verbose=False, device=DEVICE, max_det=1)
    img_w, img_h, _ = img.shape
    bbox = [0, 0, img_w, img_h]
    conf = 0.0
    
    for result in results:
        _bbox = [0, 0, img_w, img_h]
        # _label = "albopictus"
        _conf = 0.0

        bboxes_tmp = result.boxes.xyxy.tolist()
        # labels_tmp = result.boxes.cls.tolist()
        confs_tmp = result.boxes.conf.tolist()

        for bbox_tmp, conf_tmp in zip(bboxes_tmp, confs_tmp):
            if conf_tmp > _conf:
                _bbox = bbox_tmp
                _conf = conf_tmp

        if _conf > conf:
            bbox = _bbox
            # label = _label
            conf = _conf

    bbox = [int(float(mcb)) for mcb in bbox]

    return bbox, conf

In [23]:
def get_multiBbox(img, yolo_model):
    # detect mosquito
    results = yolo_model(img, verbose=False, device=DEVICE, max_det=1)
    img_w, img_h, _ = img.shape
    bbox = [0, 0, img_w, img_h]
    conf = 0.0
    bboxes = []
    confs = []
    
    for result in results:
        _bbox = [0, 0, img_w, img_h]
        # _label = "albopictus"
        _conf = 0.0

        bboxes_tmp = result.boxes.xyxy.tolist()
        # labels_tmp = result.boxes.cls.tolist()
        confs_tmp = result.boxes.conf.tolist()

        for bbox_tmp, conf_tmp in zip(bboxes_tmp, confs_tmp):
            bboxes.append([int(float(coord)) for coord in bbox_tmp])
            confs.append(conf_tmp)

    return bboxes, confs

In [11]:
# santiy check test set
for i in test_set:
    #print(batch[0].shape)
    # print(batch[1])
    print(i[2])
    break

(0.0, 0.0, 394.0, 394.0)


In [25]:
ious = []
confs = []
trues = []
preds = []
failed_dir = 'failed_imgs'
img_transform = transforms.ToPILImage()
#won_dir = 'won_imgs'
os.makedirs(failed_dir, exist_ok=True)
threshold = 0.5
idx = 0
for batch in test_set:
    img = batch[0]
    y_true = batch[1]
    bbox_true = batch[2]

    bbox, conf = get_multiBbox(img, yolo_model)
    # iou = calculate_iou(bbox, bbox_true)
    # print(bbox)
    # print(len(bbox))
    # if(idx == 100):
    #     break
    # idx+=1
    if len(bbox) != 1:
        print(f"Incorrect number of objects ({len(bbox)}) detected.")
        img = img_transform(img)
        img.save(os.path.join(failed_dir, f"image_{idx}.png"))
        idx += 1
        break
    
    # ious.append(calculate_iou(bbox, bbox_true))
    # confs.append(conf)    

Incorrect number of objects (0) detected.


In [ ]:
import numpy as np
np.mean(ious), np.mean(confs)